<a href="https://colab.research.google.com/github/Epilef86/DNC/blob/main/Data_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Qualidade dos dados. Um das ferramentas usada é o PyDeequ, objetivo de executar uma qualidade de dados dentro do nosso dataset.

A arquitetura do PyDeequ, temos 04 arqiteturas principais:

1- Computação de Métricas

Profile e Analyzers: que praticamente são algumas análises que passamos em cima do dataset que vão definir se tem qualidade ou não e ele também salva essas metrica no local que definirmos.

2- Constraint Suggestion

Ele vai olhar pro dataset e vai sugerir algumas métricas de qualidade, as vezes se eu não conheço bem a base de dados, posso usar esse método pra que o prórpio Deequ fazer essa análise


3- Constraint Verification

Performa a qualidade de dados em cima do dataset, então verificar se a coluna so tem o tipo de dado data, então eu consigo criar através desse método 

4- Metrics Repository

Aonde permite a persistência das métricas em algum lugar, colocando essa métricas em um repositório


O Deequ funciona em cima do spark, a analise de qualidade é rápida. Ele pode salvar as métricas no HDFS, AWS S3, na pórpira memória. Com isso eu posso reportes diários, mensais. 


Instalando a versão 3.0 do PySpark

In [10]:
!pip install pyspark==3.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 110.1/209.1 MB 118.5 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.8/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.8/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.8/ssl.py", line 1241, in recv_into
    return self.rea

In [7]:
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip 

In [8]:
!./ngrok authtoken 2Lh7Zu0kns54bTSQCTrDDYkyvNH_71s7E3phX7wCyU4nuAFpH
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://d9dd-34-80-60-76.ngrok.io


In [9]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pydeequ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from pyspark.sql import SparkSession

import pydeequ

spark = (
    SparkSession.builder
    .config('spark.ui.port', '4050')
    .config('spark.jars.packages', pydeequ.deequ_maven_coord)
    .config('spark.jars.excludes', pydeequ.f2j_maven_coord)
    .appName("SparkSQL")
    .getOrCreate()
)

ModuleNotFoundError: ignored

Usando o mesmo arquilo do sparkML

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType 
from pyspark.sql.functions import col

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente ', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', StringType()),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'drive/MyDrive/Colab Notebooks/case_final.json'

df = spark.read.json(
    caminho_json,
    schema = schema_base_pix,
    timestampFormat = "yyyy-MM-dd HH:mm:ss"
)

O Deequ não trabalha com estruturas, então precisamos fazer o flat desses dados

In [ ]:
df = df.withColumn(
    'destinatario_nome', col('destinatario').getField('nome')
    ).withColumn(
    'destinatario_banco', col('destinatario').getField('banco')
    ).withColumn(
    'destinatario_tipo', col('destinatario').getField('tipo')
    ).withColumn(
    'remetente_nome', col('remetente').getField('nome')
    ).withColumn(
    'remetente_banco', col('remetente').getField('banco')
    ).withColumn(
    'remetente_tipo', col('remetente').getField('tipo')
).drop('remetente', 'destinatario')

In [ ]:
nto alguns analiyzers

In [ ]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, ApproxCountDistinct, Completeness, Compliance, Mean, Size

Analisar coluna que ID_transação precisa ser completa, pra analisar isso dentro do codigo, precisamos criar o retorno de nalises que vai ser a variavel que vai guardar  resultado da analise

In [ ]:
analisisResult = (
    AnalysisRunner(spark).onData(df)
    .addAnalyzer(Size())
    .addAnalyzer(Completeness('id_transacao'))
    .addAnalyzer(Compliance('valor','valor > 0'))
    .run()
)

Pra printar precisamos transformar pra um dataframe

In [ ]:
analysisResult_df = AnalyzerContext.sucessMetricsAsDataFrame(spark,analysisResult)

In [ ]:
analysisResult_df.show()

Agora podemos ver quais foram os resultados dessas execuções de qualidade. 
Por exemplo Size vimos que o nosso dataset tem 100000 linhas
id_transacao 1.0 ou seja completo, 100% de casos que o ID está preenchido

Complice 0.99972, ou seja, esse é a porcentagem que a condição valor > 0, determinada anteriormente é certo.

Digamos que eu n consiga descrever todos os analyzers que eu quero pq não conheco muto bem o dataset. Então o deequ fornece o ConstraintSuggestionRunner, ou seja, ele vai sugerir algumas métricas de qualidade. Para isso, precisamos passar o contexto que o spark tem, qual o daataframe e o addConstrainRule e o run é pra executar esse código.

In [ ]:
from pydeequ.suggestions import ConstrainSuggestionRunner, DEFAULT

suggestionResult = ConstrainSuggestionRunner(spark).onData(df).addConstraintRule(DEFAULT()).run()

Na proxima linha foi eito um for loop que vai pegar linha a linha das susgestões que ele der e a gente printar qual foi a susgetão e qual o codigo spark já que ele nos retorna executar a qualidade.

Podermos verifica que nunca esta nulo o campo, já podemos adicionar .isComplete 

Podemos adicionar um check especifico no df, ou seja, ele vai checar se o df vai dar erro ou se nçai vai continuar Pra isso precisamos criar duas variaveis. Primeiro check ue tem nivel warning, ou seja se ele ver que está com warning e não para o programa, quando e=é error é que não pode acontecer, ele para o programa

In [ ]:
from pydeequ.checks import Check, CheckLevels, ConstrainableDataTypes
from pydeequ.verification import VerificationResult, VerificationSuite

check = Check(spark, CheckLevel.Warning, "Review Check")
error = Check(spark, CheckLevel.Error, 'Error')

Podemos cria um check result, nos temos que passar a sessão spark, qual dataframe que vai ser analisado e adicionar um check especifico. Perceba que eu estou criando atraves da variavel check, ele so vai falar pra prestar atencao aos dados mas não vai retornar uma excessao. Então eu to falando que id_trsacao seja um inteiro, não pode vim string, não pode vim double, também n posso ter id_trsacao negativa, não posso ter valo nulo

In [ ]:
checkResult = (
    VerificationSuite(spark)
    .onData(df)
    .addCheck(
        check.hasDataType('id_transacao',ConstrainableDataTypes.Integral)
        .isNonNegative('id_transacao')
        .isComplete('id_transacao')
    )
  .run()
)

Agora podemos verificar se o data frame se realmente tem a metrica de qualidade através da função VerificationResult, CheckResultAsDataframe, ele já vai retorna o resultado como dataframe e ele vai falar que o Review check está como nível warning, ele está como sucess, ou seja, nao deu falhas, ou seja, todos os tipos de id_trasacao sao realmente inteiros, não sao negativos e nao nulos

In [ ]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

Agora digamos que a gente queira colocar um erro proposital

In [ ]:
checkResult = (
    VerificationSuite(spark)
    .onData(df)
    .addCheck(
        error
          .isContainedIn('remetente_tipo', ['CNPJ'])
    )
  .run()
)

In [ ]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

O resultado deu erro pq o valor não está de acordo com o que foi passado, era pra ter apenas CNPJ, porém vem do tipo PJ ou PF. Ele vai me falar qual o erro e o que devo fazer